In [4]:


import tensorflow as tf
import sys
sys.path.append("./subjects/")
import time
import random
import tensorflow_probability as tfb
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score
from adf_utils.config import compas
from adf_data.compas import compas_data
le = LabelEncoder()
data = {'compas':compas_data}
data_config = {'compas' :compas}


dataset='compas'
if dataset == "redcar":
    sensitive_param =2
elif  dataset == "adult":
    sensitive_param =8
elif  dataset == "compas":
    sensitive_param =1

if dataset == "redcar" and sensitive_param == 2:
    sensitive_name = "race"
    priviliged_group = [{sensitive_name: 1.0}]  #male
    unpriviliged_group = [{sensitive_name: 0.0}]  #female
    favorable_label  = 1.0
    unfavorable_label = 0.0
elif dataset == "adult" and sensitive_param == 8:
    sensitive_name = "gender"
    priviliged_group = [{sensitive_name: 1.0}]  #male
    unpriviliged_group = [{sensitive_name: 0.0}]  #female
    favorable_label  = 1.0
    unfavorable_label = 0.0  
elif dataset == "compas" and sensitive_param == 1:
    sensitive_name = "race"
    priviliged_group = [{sensitive_name: 1.0}]  #Caucasian
    unpriviliged_group = [{sensitive_name: 0.0}]  #others
    favorable_label  = 1.0
    unfavorable_label = 0.0  
X, Y, input_shape, nb_classes = data[dataset]()

df = pd.DataFrame(X)
df.columns = data_config[dataset]().feature_name
A = df[sensitive_name].to_numpy()
X = df.to_numpy()

df['label'] = Y

X_train, X_test, y_train, y_test, A_train, A_test = train_test_split(X, Y, A, random_state=0,stratify= Y)
           
len_df = df.shape[0]
df_out=pd.read_csv('./Results/LogisticRegression/compas/race/compas_coef_NI_1000.csv')
coef_list = ['ry', 'rd', 'rp', 'rj','sy', 'sd', 'sp', 'sj', 'ay', 'ad', 'ap', 'aj', 
             'jp', 'jd', 'jy', 'pd','dy']

coef_mean = df_out[df_out.columns[8:]].mean()

# step = -0.001
# for i in range(500):
#     direction = np.random.choice([2,-2],size=len(coef_list))
edges = {}
for i in range(len(coef_mean)):
    #print(coef_mean[i])
    edges[coef_list[i]] = coef_mean[i]# +  direction[i] * step
#         #print(edges[coef_list[i]])

#     #print('zie',edges['z1e'])
remove_edge =None #'jy'

num_samples =len_df


models =[]
dfs =[]
acc =[]
f1  =[]
test_bal =[]
pred_bal =[]

for i in range(100):

    r_prob = df['race'].sum()/df.shape[0]
    r = tfb.distributions.Bernoulli(r_prob).sample(num_samples).numpy()

    s_prob = df['sex'].sum()/df.shape[0]
    s = tfb.distributions.Bernoulli(s_prob).sample(num_samples).numpy()

    a_pop = df['age_cat'].unique()
    a_prob=[]
    for age in a_pop:
        a_prob.append(np.where(df['age_cat']==age)[0].shape[0]/df.shape[0])
    a=np.array(random.choices(a_pop, weights=a_prob, k=num_samples))

#     j[i] ~ poisson(exp((rj * r[i]) + (sj * s[i]) + (aj * a[i])) ); 
    j_rate = tf.exp((edges['rj'] * r) + (edges['sj'] * s) + (edges['aj'] * a) )
    j =  tfb.distributions.Poisson(rate=j_rate).sample().numpy() 
    
#     p[i] ~ poisson(exp((rp * r[i]) + (sp * s[i]) + (ap * a[i]) + (jp * j[i])));
    p_rate = tf.exp((edges['rp'] * r) + (edges['sp'] * s) + (edges['ap'] * a) + (edges['jp'] * j))
    p =  tfb.distributions.Poisson(rate=p_rate).sample().numpy() 
    
#    d[i] ~ poisson(exp( (rd * r[i]) + (sd * s[i]) + (ad * a[i]) + (pd * p[i]) + (jd * j[i]) )); 
    d_rate = tf.exp((edges['rd'] * r) + (edges['sd'] * s) + (edges['ad'] * a) + (edges['jd'] * j) + (edges['pd'] * p) )
    d =  tfb.distributions.Poisson(rate=d_rate).sample().numpy() 
    
#    y[i] ~ bernoulli_logit( (ry * r[i]) + (sy * s[i]) + (ay * a[i]) + (dy * d[i]) + (jy * j[i]) );


    y_logits = (edges['ry'] * r) + (edges['sy'] * s) + (edges['ay'] * a) + (edges['dy'] * d) + (edges['jy'] * j)
    y = tfb.distributions.Bernoulli(logits = y_logits).sample().numpy()
    
    if y.sum()/len(y)<0.2 or y.sum()/len(y)>0.85:
        print('Bal', y.sum()/len(y))
        continue
    new_df = pd.DataFrame(columns = df.columns)
    new_df['sex'] = s
    new_df['race'] = r
    new_df['age_cat'] = a
    new_df['priors_count'] = p
    new_df['juv_fel_count'] = j
    new_df['r_charge_degree'] = d
    new_df['label'] = y
    if new_df.isna().sum().sum()!=0 or np.isinf(new_df).sum().sum()!=0:
        print('NA')
        continue

    X = new_df.to_numpy()[:,:-1]
    Y = new_df.to_numpy()[:,-1]
    X_train, X_test, y_train, y_test = train_test_split(X, Y,stratify= Y)
    model = LogisticRegression()
    model.fit(X_train,y_train)
    preds = model.predict(X_test)
    acc_temp = accuracy_score(y_test,preds)
    f1_temp = f1_score(y_test,preds)

    print(round(acc_temp,2),round(f1_temp,2))
    input()
    if round(acc_temp,2) >0.9 and  round(f1_temp,2)>0.80 :
    #if True:
        print('Prediction balance = ',preds.sum()/len(preds), 'test balance = ', y_test.sum()/len(y_test) )

        print(i,acc_temp, f1_temp)

        #print(model.score(X_test,y_test), accuracy_score(y_test,preds))
        f1.append(round(f1_temp,2))
        models.append(model)
        dfs.append(new_df.copy())

        test_bal.append(y_test.sum()/len(y_test))
        pred_bal.append(preds.sum()/len(preds))


# df_out_2 = pd.DataFrame([acc,f1,test_bal,pred_bal]).T
#dfs[0].to_csv('./subjects/datasets/adult_pp.csv',index=False)

0.85 0.92

0.84 0.91

0.84 0.92

0.85 0.92

0.85 0.92

0.85 0.92

0.84 0.91

0.84 0.91

Bal 0.8508455780426948
0.85 0.92

0.83 0.91

0.84 0.91

0.85 0.92

0.84 0.91

0.84 0.91

0.84 0.91

0.85 0.92

0.84 0.91

0.84 0.92

0.84 0.92

0.84 0.91

0.84 0.91


KeyboardInterrupt: Interrupted by user

In [1]:
new_df

NameError: name 'new_df' is not defined

In [ ]:
import tensorflow as tf
import sys
sys.path.append("./Fairness-libraries-testing/")
import time
import random
import tensorflow_probability as tfb
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score

le = LabelEncoder()
df = pd.read_csv('../subjects/datasets/adult_pp.csv')

models =[]
dfs =[]
acc =[]
f1  =[]
test_bal =[]
pred_bal =[]

for col in [ 'workclass', 'educational-num', 'marital-status', 'occupation',
       'relationship',  'hours-per-week', 'native-country',
       'label']:
    print(col)
    for i in range(10):
        df[col] = np.random.choice(df[col],size = df.shape[0])


        X = df.to_numpy()[:,:-1]
        Y = df.to_numpy()[:,-1]
        X_train, X_test, y_train, y_test = train_test_split(X, Y,stratify= Y)
        model = LogisticRegression()
        model.fit(X_train,y_train)
        preds = model.predict(X_test)
        acc_temp = accuracy_score(y_test,preds)
        f1_temp = f1_score(y_test,preds)
        print(i,acc_temp, f1_temp)
        if round(acc_temp,2) >=0.9 and  round(f1_temp,2)>0.85 :
        #if True:
            print('Prediction balance = ',preds.sum()/len(preds), 'test balance = ', y_test.sum()/len(y_test) )

            print(i,acc_temp, f1_temp)

            #print(model.score(X_test,y_test), accuracy_score(y_test,preds))
            f1.append(round(f1_temp,2))
            models.append(model)
            dfs.append(new_df.copy())

            test_bal.append(y_test.sum()/len(y_test))
            pred_bal.append(preds.sum()/len(preds))
            input()

# df_out_2 = pd.DataFrame([acc,f1,test_bal,pred_bal]).T
#dfs[0].to_csv('./subjects/datasets/adult_pp.csv',index=False)

In [23]:
# age, sex, education level, marital status, work-class and number
# of working hours per week.
import tensorflow as tf
import tensorflow_probability as tfp
import sys
sys.path.append("./Fairness-libraries-testing/")
import time
import random
import tensorflow_probability as tfb
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score
# np.random.seed(0)
# tf.random.set_seed(0)
le = LabelEncoder()
df = pd.read_csv('../subjects/datasets/adult_pp.csv')
len_df = df.shape[0]
num_samples = df.shape[0]
z2 = df['age']
x = df['gender']
z1 = df['race']
n = df['native-country']

models =[]
dfs =[]
acc =[]
f1  =[]
test_bal =[]
pred_bal =[]

acc_max = 0
for i in range(100000):
    z1e = tfb.distributions.Normal(loc=0, scale=10).sample().numpy()
    z2e = tfb.distributions.Normal(loc=0, scale=10).sample().numpy()
    ne = tfb.distributions.Normal(loc=0, scale=10).sample().numpy()
    xe = tfb.distributions.Normal(loc=0, scale=10).sample().numpy()
    e0 = tfb.distributions.Normal(loc=0, scale=10).sample().numpy()
    
    eh = tfb.distributions.Normal(loc=0, scale=10).sample().numpy()
    z1h = tfb.distributions.Normal(loc=0, scale=10).sample().numpy()
    z2h = tfb.distributions.Normal(loc=0, scale=10).sample().numpy()
    nh = tfb.distributions.Normal(loc=0, scale=10).sample().numpy()
    xh = tfb.distributions.Normal(loc=0, scale=10).sample().numpy()
    h0 = tfb.distributions.Normal(loc=0, scale=10).sample().numpy()
    
    ew = tfb.distributions.Normal(loc=0, scale=10).sample().numpy()
    z2w = tfb.distributions.Normal(loc=0, scale=10).sample().numpy()
    nw = tfb.distributions.Normal(loc=0, scale=10).sample().numpy()
    hw = tfb.distributions.Normal(loc=0, scale=10).sample().numpy()
    w0 = tfb.distributions.Normal(loc=0, scale=10).sample().numpy()
    
    
    z1m = tfb.distributions.Normal(loc=0, scale=10).sample().numpy()
    z2m = tfb.distributions.Normal(loc=0, scale=10).sample().numpy()
    nm = tfb.distributions.Normal(loc=0, scale=10).sample().numpy()
    xm = tfb.distributions.Normal(loc=0, scale=10).sample().numpy()
    hm = tfb.distributions.Normal(loc=0, scale=10).sample().numpy()
    wm = tfb.distributions.Normal(loc=0, scale=10).sample().numpy()
    m0 = tfb.distributions.Normal(loc=0, scale=10).sample().numpy()
    
    z1o = tfb.distributions.Normal(loc=0, scale=10).sample().numpy()
    z2o = tfb.distributions.Normal(loc=0, scale=10).sample().numpy()
    eo = tfb.distributions.Normal(loc=0, scale=10).sample().numpy()
    xo = tfb.distributions.Normal(loc=0, scale=10).sample().numpy()
    mo = tfb.distributions.Normal(loc=0, scale=10).sample().numpy()
    wo = tfb.distributions.Normal(loc=0, scale=10).sample().numpy()
    o0 = tfb.distributions.Normal(loc=0, scale=10).sample().numpy()
    
    z2r = tfb.distributions.Normal(loc=0, scale=10).sample().numpy()
    er = tfb.distributions.Normal(loc=0, scale=10).sample().numpy()
    xr = tfb.distributions.Normal(loc=0, scale=10).sample().numpy()
    mr = tfb.distributions.Normal(loc=0, scale=10).sample().numpy()
    nr = tfb.distributions.Normal(loc=0, scale=10).sample().numpy()
    r0 = tfb.distributions.Normal(loc=0, scale=10).sample().numpy()
    
    z1y = tfb.distributions.Normal(loc=0, scale=10).sample().numpy()
    z2y = tfb.distributions.Normal(loc=0, scale=10).sample().numpy()
    ny = tfb.distributions.Normal(loc=0, scale=10).sample().numpy()
    ey = tfb.distributions.Normal(loc=0, scale=10).sample().numpy()
    hy = tfb.distributions.Normal(loc=0, scale=10).sample().numpy()
    wy = tfb.distributions.Normal(loc=0, scale=10).sample().numpy()
    my = tfb.distributions.Normal(loc=0, scale=10).sample().numpy()
    oy = tfb.distributions.Normal(loc=0, scale=10).sample().numpy()
    ry = tfb.distributions.Normal(loc=0, scale=10).sample().numpy()
    y0 = tfb.distributions.Normal(loc=0, scale=10).sample().numpy()
    
#     z1e = tfp.distributions.Uniform(0,10).sample().numpy()
#     z2e = tfp.distributions.Uniform(0,10).sample().numpy()
#     ne = tfp.distributions.Uniform(0,10).sample().numpy()
#     xe = tfp.distributions.Uniform(0,10).sample().numpy()
    
#     eh = tfp.distributions.Uniform(0,10).sample().numpy()
#     z1h = tfp.distributions.Uniform(0,10).sample().numpy()
#     z2h = tfp.distributions.Uniform(0,10).sample().numpy()
#     nh = tfp.distributions.Uniform(0,10).sample().numpy()
#     xh = tfp.distributions.Uniform(0,10).sample().numpy()
    
#     ew = tfp.distributions.Uniform(0,10).sample().numpy()
#     z2w =tfp.distributions.Uniform(0,10).sample().numpy()
#     nw =tfp.distributions.Uniform(0,10).sample().numpy()
#     hw = tfp.distributions.Uniform(0,10).sample().numpy()
    
    
#     z1m = tfp.distributions.Uniform(0,10).sample().numpy()
#     z2m = tfp.distributions.Uniform(0,10).sample().numpy()
#     nm = tfp.distributions.Uniform(0,10).sample().numpy()
#     xm =tfp.distributions.Uniform(0,10).sample().numpy()
#     hm = tfp.distributions.Uniform(0,10).sample().numpy()
#     wm = tfp.distributions.Uniform(0,10).sample().numpy()
    
#     z1o = tfp.distributions.Uniform(0,10).sample().numpy()
#     z2o = tfp.distributions.Uniform(0,10).sample().numpy()
#     eo = tfp.distributions.Uniform(0,10).sample().numpy()
#     xo = tfp.distributions.Uniform(0,10).sample().numpy()
#     mo =tfp.distributions.Uniform(0,10).sample().numpy()
#     wo = tfp.distributions.Uniform(0,10).sample().numpy()
    
#     z2r = tfp.distributions.Uniform(0,10).sample().numpy()
#     er = tfp.distributions.Uniform(0,10).sample().numpy()
#     xr = tfp.distributions.Uniform(0,10).sample().numpy()
#     mr =tfp.distributions.Uniform(0,10).sample().numpy()
#     nr = tfp.distributions.Uniform(0,10).sample().numpy()
    
#     z1y = tfp.distributions.Uniform(0,10).sample().numpy()
#     z2y = tfp.distributions.Uniform(0,10).sample().numpy()
#     ny = tfp.distributions.Uniform(0,10).sample().numpy()
#     ey = tfp.distributions.Uniform(0,10).sample().numpy()
#     hy = tfp.distributions.Uniform(0,10).sample().numpy()
#     wy = tfp.distributions.Uniform(0,10).sample().numpy()
#     my = tfp.distributions.Uniform(0,10).sample().numpy()
#     oy = tfp.distributions.Uniform(0,10).sample().numpy()
#     ry = tfp.distributions.Uniform(0,10).sample().numpy()
    
    x_prob = df['gender'].sum()/df.shape[0]
    x = tfb.distributions.Bernoulli(x_prob).sample(num_samples).numpy()

    z1_pop = df['race'].unique()
    z1_prob=[]
    for race in z1_pop:
        z1_prob.append(np.where(df['race']==race)[0].shape[0]/df.shape[0])
    z1=np.array(random.choices(z1_pop, weights=z1_prob, k=num_samples))

    z2_pop = df['age'].unique()
    z2_prob=[]
    for age in z2_pop:
        z2_prob.append(np.where(df['age']==age)[0].shape[0]/df.shape[0])
    z2=np.array(random.choices(z2_pop, weights=z2_prob, k=num_samples))

    n_pop = df['native-country'].unique()
    n_prob=[]
    for nc in n_pop:
        n_prob.append(np.where(df['native-country']==nc)[0].shape[0]/df.shape[0])

    n =np.array(random.choices(n_pop, weights=n_prob, k=num_samples))

    e_rate = tf.exp(e0 + (z1e * z1) + (z2e * z2) + (ne * n) + (xe * x))
    e =  tfb.distributions.Poisson(rate=e_rate).sample().numpy() 
    input()
    h_mean = h0 + (eh * e) + (z1h * z1) + (z2h * z2) + (nh * n) + (xh * x)
    h = tfb.distributions.Normal(loc=h_mean, scale= 1).sample().numpy()
    
    
    w_rate = tf.exp(w0 + (ew * e) + (z2w * z2) + (nw * n) + (hw * h))
    w =  tfb.distributions.Poisson(rate=w_rate).sample().numpy()
    

    m_rate = tf.exp(m0 + (z1m * z1) + (z2m * z2) + (nm * n) + (xm * x) + (hm * h) + (wm * w))
    m =  tfb.distributions.Poisson(rate=m_rate).sample().numpy()
    
    o_rate = tf.exp(o0 + (z1o * z1) + (z2o * z2) + (eo * e) + (xo * x) + (mo * m) + (wo * w))
    o =  tfb.distributions.Poisson(rate=o_rate).sample().numpy()
    
    r_rate = tf.exp(r0 + (z2r * z2) + (er * e) + (xr * x) + (mr * m) + (nr * n))
    r =  tfb.distributions.Poisson(rate=r_rate).sample().numpy()

    y_logits =y0 + (z1y * z1) + (z2y * z2) + (ny * n) + (ey * e) + (hy * h) + (wy * w) + (my * m) + (oy * o) + (ry * r)
    y = tfb.distributions.Bernoulli(logits = y_logits).sample().numpy()
    if y.sum()/len_df<0.2 or y.sum()/len_df>0.8:
        continue
    
    
    new_df = pd.DataFrame(columns = df.columns)
    new_df['age'] = z2
    new_df['workclass'] = w
    new_df['educational-num'] = e
    new_df['marital-status'] = m
    new_df['occupation'] = o
    new_df['relationship'] = r
    new_df['race'] = z1
    new_df['gender'] = x
    new_df['hours-per-week'] = h
    new_df['native-country'] = n
    new_df['label'] = y
    if new_df.isna().sum().sum()!=0 or np.isinf(new_df).sum().sum()!=0:
        #print('NA')
        continue
    X = new_df.to_numpy()[:,:-1]
    Y = new_df.to_numpy()[:,-1]
    X_train, X_test, y_train, y_test = train_test_split(X, Y,stratify= Y)
    model = LogisticRegression()
    model.fit(X_train,y_train)
    preds = model.predict(X_test)
    acc_temp = accuracy_score(y_test,preds)
    f1_temp = f1_score(y_test,preds)
    if acc_temp > acc_max:
        acc_max = acc_temp
        print('Accuracy ', acc_temp,'  F1', f1_temp)
    print('ACC , F1',acc_temp, f1_temp ,preds.sum()/preds.shape[0] )
    if round(acc_temp,2) >0.89 and round(acc_temp,2) <0.94 and  round(f1_temp,2)>0.85  and round(f1_temp,2)<0.94 :
    #if True:
        print('Prediction balance = ',preds.sum()/len(preds), 'test balance = ', y_test.sum()/len(y_test) )

        print(i,acc_temp, f1_temp)

        #print(model.score(X_test,y_test), accuracy_score(y_test,preds))
        f1.append(round(f1_temp,2))
        models.append(model)
        dfs.append(new_df.copy())

        test_bal.append(y_test.sum()/len(y_test))
        pred_bal.append(preds.sum()/len(preds))
        

KeyboardInterrupt: Interrupted by user

In [24]:
import tensorflow as tf
import sys
sys.path.append("./subjects/")
import time
import random
import tensorflow_probability as tfb
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score
from adf_utils.config import census
from adf_data.census import census_data
X, Y, input_shape, nb_classes = census_data()

df = pd.DataFrame(X)
df.columns = census().feature_name
Y = np.argmax(Y,axis=1)
df['label']= Y
X_train, X_test, y_train, y_test = train_test_split(X, Y,stratify= Y)
model = LogisticRegression()
model.fit(X_train,y_train)
preds = model.predict(X_test)
acc_temp = accuracy_score(y_test,preds)
f1_temp = f1_score(y_test,preds)

/home/vmonjezi/.virtualenvs/RISC_LAB/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


(0.8080088441223436, 0.4836471754212091)

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [0., 1.]])

In [64]:
X

array([[ 1.02202860e+00,  4.11660348e+00,  3.58804674e+00,
         2.36702243e+00, -1.37195659e+00,  3.29604478e+00,
        -2.13604152e+00,  5.85466355e+00,  8.92534306e+00,
        -2.17482065e+00],
       [ 2.20656076e+00,  5.50616718e+00,  1.66794070e+00,
         5.95360909e-01, -2.57545698e+00,  1.49786432e+00,
        -2.95452597e+00,  9.78623541e+00,  8.76360303e+00,
        -2.76924393e+00],
       [ 1.04278730e+00,  4.60625923e+00,  1.42094543e+00,
         5.34922494e-01, -2.19936446e+00,  2.55832910e+00,
        -2.06140206e+00,  6.10917741e+00,  9.45068135e+00,
        -2.73295056e+00],
       [ 2.47034915e+00,  4.09862906e+00,  2.36833522e+00,
         4.35679206e-02, -4.07989383e+00,  3.57150086e+00,
        -3.83819576e-01,  7.09329500e+00,  1.15430098e+01,
        -3.78553530e+00],
       [ 5.84500078e+00,  2.36376889e+00,  1.48780331e+00,
         8.91392213e+00, -6.69612814e+00, -9.60517307e+00,
        -1.08661170e+01,  7.62179362e+00,  4.39001161e+00,
         9.

In [65]:
y

array([0, 0, 0, 0, 1, 1, 1, 1, 0, 1])

In [2]:
from adf_data.adult import adult_data
from adf_utils.config import adult

ModuleNotFoundError: No module named 'adf_data'